# 🛒 Market Basket Intelligence: From Groceries to Great Recommendations

> **Imagine** you're a data scientist at a large retail chain. Every day, thousands of customers buy items together —
> tea with biscuits, pasta with sauce, or maybe chocolate with everything. Hidden in those baskets are **patterns** 
> that can make recommendations smarter, shelves better arranged, and promotions more profitable.  
>
> In this notebook, we’ll turn a raw transaction log into **actionable insights** using Market Basket Analysis, 
> uncovering which products **love to be bought together** and how that knowledge can power **recommendation systems**.
>
> **Dataset**: `Market_Basket_Optimisation.csv` — a classic transactional dataset where each row represents a single
> shopping basket with items spread across columns.
>
> **Problem Type**: Unsupervised learning → **Association Rule Mining** (Apriori / FP-Growth) + simple **recommender evaluation**.
>
> **Main Questions**  
> 1. Which items are most frequently purchased?  
> 2. What item combinations (pairs/triads) commonly occur together?  
> 3. Which association rules (X ⇒ Y) are the strongest by **support**, **confidence**, and **lift**?  
> 4. Can these rules help **recommend** the next item in a basket?
>
> ### ✨ What You’ll See
> - Clean, reproducible EDA with clear visualizations.  
> - Association rules through Apriori (with a built‑in fallback if libraries aren’t available).  
> - Side‑by‑side comparison of algorithms and rule quality.  
> - Lightweight evaluation of recommendations using hold‑out baskets.
>
> Let’s dive in! 🚀


## 1. Introduction / Problem Statement ✍️

## 2. Import Libraries + Basic Setup 🧰

In [ ]:
# Basic setup
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import math
import time
import random
import itertools
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Matplotlib defaults (keep simple and clean; no explicit colors/styles)
plt.rcParams['figure.figsize'] = (9, 5)
plt.rcParams['axes.grid'] = True

DATA_PATH = "/mnt/data/Market_Basket_Optimisation.csv"
assert os.path.exists(DATA_PATH), f"Dataset not found at {DATA_PATH}"


## 3. Dataset Loading + First Look 👀

In [ ]:
# Load dataset
df_raw = pd.read_csv(DATA_PATH, header=None)
df_raw.head()


In [ ]:
# Basic overview
print("Shape:", df_raw.shape)
print("\nInfo:")
df_raw.info()
print("\nDescribe (non-numeric will be skipped or shown as object count):")
display(df_raw.describe(include='all'))


In [ ]:
# Missing values & duplicates check
missing_count = df_raw.isna().sum().sum()
print("Total missing values:", missing_count)

# Each row is a basket; duplicates at row-level are acceptable in transactional logs,
# but we still check exact duplicate rows:
dupe_rows = df_raw.duplicated().sum()
print("Duplicate rows:", dupe_rows)

# Transform to list-of-lists (transactions), dropping NaNs/empties, and trimming whitespace
transactions = []
for _, row in df_raw.iterrows():
    items = [str(x).strip() for x in row.values if isinstance(x, str) and str(x).strip() != "" and str(x).lower() != "nan"]
    if items:
        transactions.append(items)

print("Total transactions (non-empty):", len(transactions))

# Memory usage (rough estimate via DataFrame)
mem_bytes = df_raw.memory_usage(index=True).sum()
print(f"Approx memory usage: {mem_bytes/1024:.2f} KB")


## 4. Exploratory Data Analysis (EDA) 🔎

In [ ]:
# Item frequencies
all_items = list(itertools.chain.from_iterable(transactions))
item_counts = Counter(all_items)
item_freq = pd.DataFrame(item_counts.items(), columns=["item", "count"]).sort_values("count", ascending=False)
item_freq.head(10)

In [ ]:
# Plot: Top 20 items
top_k = 20
top_items = item_freq.head(top_k)

plt.figure()
plt.bar(top_items["item"], top_items["count"])
plt.xticks(rotation=75, ha='right')
plt.title(f"Top {top_k} Most Frequent Items")
plt.xlabel("Item")
plt.ylabel("Count")
plt.tight_layout()
plt.show()
print("These are the most frequently purchased items across all baskets. Higher bars indicate items that appear in more transactions.")

In [ ]:
# Basket length distribution
basket_lengths = [len(t) for t in transactions]

plt.figure()
plt.hist(basket_lengths, bins=range(1, max(basket_lengths)+2))
plt.title("Basket Size Distribution")
plt.xlabel("Number of items in a basket")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()
print("Most baskets contain a small number of items. This distribution helps set expectations for rule sizes and support thresholds.")

In [ ]:
# Unique items and basic stats
n_transactions = len(transactions)
n_unique_items = item_freq.shape[0]
avg_basket_size = np.mean(basket_lengths)

print(f"Unique items: {n_unique_items}")
print(f"Average basket size: {avg_basket_size:.2f}")
print(f"Median basket size: {np.median(basket_lengths):.0f}")

In [ ]:
# Pair (2-item) co-occurrence counts (top 20)
pair_counts = Counter()
for t in transactions:
    for a, b in itertools.combinations(sorted(set(t)), 2):
        pair_counts[(a, b)] += 1

pair_df = pd.DataFrame([{"item_a": a, "item_b": b, "count": c} for (a, b), c in pair_counts.items()])
pair_df = pair_df.sort_values("count", ascending=False).head(20)
pair_df

In [ ]:
# Plot: Top 20 item pairs by co-occurrence
plt.figure()
plt.bar([f"{a} & {b}" for a,b in zip(pair_df["item_a"], pair_df["item_b"])], pair_df["count"])
plt.xticks(rotation=75, ha='right')
plt.title("Top 20 Item Pairs by Co-occurrence")
plt.xlabel("Item Pair")
plt.ylabel("Count")
plt.tight_layout()
plt.show()
print("These item pairs co-occur most often in the same basket. They are candidates for strong association rules.")

## 5. Data Cleaning / Preprocessing 🧹

In [ ]:
# One-hot encode transactions for association mining
# Build item index
items_sorted = sorted(item_freq['item'].tolist())
item_to_idx = {item:i for i, item in enumerate(items_sorted)}

# Sparse-like construction into a DataFrame
rows = []
for t in transactions:
    row = [0]*len(items_sorted)
    for it in set(t):  # set to avoid double count within a basket
        row[item_to_idx[it]] = 1
    rows.append(row)

basket_ohe = pd.DataFrame(rows, columns=items_sorted)
basket_ohe.shape

In [ ]:
# Simple feature engineering ideas (illustrative, not always needed for ARM):
# 1) Basket size as a feature (for recommendation diagnostics)
# 2) Weekend vs weekday purchases (if timestamps existed; here we don't have them)
# We'll keep basket size vector for later evaluation slices.
basket_sizes = pd.Series(basket_lengths, name="basket_size")
basket_sizes.describe()

## 6. Model Building (Baseline + Advanced) 🧠

In [ ]:
# We will mine frequent itemsets and generate association rules.
# Try mlxtend if available; otherwise use a minimal Apriori fallback.

def generate_rules_from_itemsets(freq_itemsets, min_confidence=0.2):
    """Given dict itemset->support_count and total transaction count,
    compute rules with confidence & lift."""
    n_trans = len(transactions)
    support = {itemset:cnt/n_trans for itemset, cnt in freq_itemsets.items()}
    rules = []
    for itemset, cnt in freq_itemsets.items():
        if len(itemset) < 2:
            continue
        items = list(itemset)
        for r in range(1, len(items)):
            for antecedent in itertools.combinations(items, r):
                consequent = tuple(sorted(set(items) - set(antecedent)))
                if not consequent:
                    continue
                antecedent = tuple(sorted(antecedent))
                sup_itemset = support[itemset]
                sup_ante = support.get(antecedent, 0)
                sup_cons = support.get(consequent, 0)
                if sup_ante == 0 or sup_cons == 0:
                    continue
                conf = sup_itemset / sup_ante
                lift = conf / sup_cons if sup_cons > 0 else np.nan
                if conf >= min_confidence:
                    rules.append({
                        "antecedent": antecedent,
                        "consequent": consequent,
                        "support": sup_itemset,
                        "confidence": conf,
                        "lift": lift
                    })
    return pd.DataFrame(rules).sort_values(["confidence","lift","support"], ascending=False)

def apriori_fallback(transactions, min_support=0.01, max_len=3):
    n_trans = len(transactions)
    # L1
    item_counts = Counter(itertools.chain.from_iterable(transactions))
    L = { (i,): c for i,c in item_counts.items() if c/n_trans >= min_support }
    freq_itemsets = dict(L)
    k = 2
    while L and k <= max_len:
        # candidate generation
        items = sorted(set(itertools.chain.from_iterable(L.keys())))
        Ck = set()
        L_keys = list(L.keys())
        for i in range(len(L_keys)):
            for j in range(i+1, len(L_keys)):
                a, b = L_keys[i], L_keys[j]
                union = tuple(sorted(set(a) | set(b)))
                if len(union) == k:
                    # prune: all (k-1)-subsets must be frequent
                    all_subs_ok = True
                    for sub in itertools.combinations(union, k-1):
                        if sub not in L:
                            all_subs_ok = False
                            break
                    if all_subs_ok:
                        Ck.add(union)
        # count candidates
        Ck_counts = Counter()
        for t in transactions:
            tset = set(t)
            for cand in Ck:
                if set(cand).issubset(tset):
                    Ck_counts[cand] += 1
        # filter by support
        L = { cand:cnt for cand,cnt in Ck_counts.items() if cnt/n_trans >= min_support }
        freq_itemsets.update(L)
        k += 1
    return freq_itemsets

# Try mlxtend
use_mlxtend = False
try:
    from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
    use_mlxtend = True
except Exception as e:
    print("mlxtend not available; using fallback Apriori.", str(e))

results = {}

if use_mlxtend:
    # Apriori
    t0 = time.time()
    fi_ap = apriori(basket_ohe.astype(bool), min_support=0.01, use_colnames=True, max_len=3)
    fi_ap["length"] = fi_ap["itemsets"].apply(len)
    rules_ap = association_rules(fi_ap, metric="confidence", min_threshold=0.2)
    t1 = time.time()
    results['Apriori'] = {
        "fi": fi_ap.sort_values("support", ascending=False),
        "rules": rules_ap.sort_values(["confidence","lift","support"], ascending=False),
        "time": t1 - t0
    }

    # FP-Growth
    t0 = time.time()
    fi_fp = fpgrowth(basket_ohe.astype(bool), min_support=0.01, use_colnames=True, max_len=3)
    fi_fp["length"] = fi_fp["itemsets"].apply(len)
    rules_fp = association_rules(fi_fp, metric="confidence", min_threshold=0.2)
    t1 = time.time()
    results['FP-Growth'] = {
        "fi": fi_fp.sort_values("support", ascending=False),
        "rules": rules_fp.sort_values(["confidence","lift","support"], ascending=False),
        "time": t1 - t0
    }
else:
    # Fallback Apriori only
    t0 = time.time()
    fi_dict = apriori_fallback(transactions, min_support=0.01, max_len=3)
    t1 = time.time()
    fi_df = pd.DataFrame(
        [ {"itemsets": set(k), "support": v/len(transactions), "length": len(k)} for k,v in fi_dict.items() ]
    ).sort_values("support", ascending=False)
    rules_ap = generate_rules_from_itemsets(fi_dict, min_confidence=0.2)
    results['Apriori (fallback)'] = {
        "fi": fi_df,
        "rules": rules_ap,
        "time": t1 - t0
    }

# Peek rules
for k,v in results.items():
    print(f"\n{k}: {len(v['fi'])} frequent itemsets, {len(v['rules'])} rules, time: {v['time']:.3f}s")
    display(v['rules'].head(10))


## 7. Model Evaluation 📏

In [ ]:
# For association rules, classic metrics are support, confidence, and lift.
# We'll also build a tiny next-item recommendation test to compare algorithms.
# Protocol: For each basket with length >= 2, hide the last item as a 'target'.
# Use rules to recommend top-K consequents based on items in the observed prefix.
# Compute Precision@K and HitRate@K.

def build_recommender_from_rules(rules_df, topn=3):
    """Create a mapping from antecedent -> list of (consequent, score) sorted by confidence*lift."""
    mapping = defaultdict(list)
    if rules_df is None or rules_df.empty:
        return mapping
    # Normalize format between mlxtend and fallback
    if 'antecedents' in rules_df.columns:
        # mlxtend format
        for _, row in rules_df.iterrows():
            ante = tuple(sorted(list(row['antecedents'])))
            cons = tuple(sorted(list(row['consequents'])))
            score = float(row.get('confidence', 0)) * float(row.get('lift', 1))
            mapping[ante].append((cons, score))
    else:
        for _, row in rules_df.iterrows():
            ante = tuple(sorted(list(row['antecedent'])))
            cons = tuple(sorted(list(row['consequent'])))
            score = float(row.get('confidence', 0)) * float(row.get('lift', 1))
            mapping[ante].append((cons, score))

    for k in mapping:
        mapping[k] = sorted(mapping[k], key=lambda x: x[1], reverse=True)[:topn]
    return mapping

def recommend(prefix_items, rule_map, K=5):
    """Aggregate recommendations from all rule antecedents that are subset of prefix."""
    prefix_set = set(prefix_items)
    cand_scores = defaultdict(float)
    for ante, cons_list in rule_map.items():
        if set(ante).issubset(prefix_set):
            for cons, score in cons_list:
                for item in cons:
                    if item not in prefix_set:
                        cand_scores[item] += score
    ranked = sorted(cand_scores.items(), key=lambda x: x[1], reverse=True)
    return [it for it,_ in ranked[:K]]

# Build train/test split by baskets
rng = np.random.default_rng(42)
indices = np.arange(len(transactions))
rng.shuffle(indices)
split = int(0.8 * len(indices))
train_idx, test_idx = indices[:split], indices[split:]

train_tx = [transactions[i] for i in train_idx]
test_tx  = [transactions[i] for i in test_idx]

# Re-mine on train only (simple)
# To keep it light, reuse earlier frequent itemsets/rules mined on all data,
# but for a stricter protocol you could re-run mining on train_tx.

def evaluate_rules(rule_df, test_baskets, K=5):
    rule_map = build_recommender_from_rules(rule_df, topn=5)
    hits = 0
    total = 0
    all_prec = []
    for t in test_baskets:
        if len(t) < 2:
            continue
        prefix = t[:-1]
        target = t[-1]
        recs = recommend(prefix, rule_map, K=K)
        if not recs:
            continue
        total += 1
        hit = 1 if target in recs else 0
        hits += hit
        prec = hit / len(recs)
        all_prec.append(prec)
    hit_rate = hits / total if total > 0 else 0.0
    mean_precision = float(np.mean(all_prec)) if all_prec else 0.0
    return hit_rate, mean_precision, total

comparison_rows = []
for name, out in results.items():
    rules_df = out['rules']
    hr, mp, n_eval = evaluate_rules(rules_df, test_tx, K=5)
    comparison_rows.append({
        "Model": name,
        "Rules": len(rules_df) if rules_df is not None else 0,
        "Time (s)": round(out['time'], 3),
        "HitRate@5": round(hr, 4),
        "MeanPrecision@5": round(mp, 4),
        "Eval Baskets": n_eval
    })

model_comparison = pd.DataFrame(comparison_rows).sort_values("HitRate@5", ascending=False)
model_comparison

In [ ]:
# Plot model performance
plt.figure()
plt.bar(model_comparison['Model'], model_comparison['HitRate@5'])
plt.title("Model HitRate@5")
plt.xlabel("Model")
plt.ylabel("HitRate@5")
plt.tight_layout()
plt.show()

plt.figure()
plt.bar(model_comparison['Model'], model_comparison['MeanPrecision@5'])
plt.title("Model MeanPrecision@5")
plt.xlabel("Model")
plt.ylabel("MeanPrecision@5")
plt.tight_layout()
plt.show()

print("We evaluate recommendation quality using hold-out baskets. HitRate@5 measures how often the hidden last item appears in the top-5 recommendations. MeanPrecision@5 penalizes longer lists if they don't include the target.")

## 9. Feature Importance + Explainability 💡

In [ ]:
# For association rules, 'explainability' comes from rule metrics:
# - Support: how common the itemset is
# - Confidence: P(Y | X) likelihood the consequent occurs given the antecedent
# - Lift: multiplicative boost over random chance; >1 is positive association

# Show top rules by lift for the best model
best_model = model_comparison.iloc[0]['Model']
best_rules = results[best_model]['rules']

# Normalize columns for display
if 'antecedents' in best_rules.columns:
    disp_rules = best_rules[['antecedents','consequents','support','confidence','lift']].head(10)
else:
    disp_rules = best_rules[['antecedent','consequent','support','confidence','lift']].head(10)

disp_rules

## 10. Conclusion + Future Work ✅

- We explored a classic market-basket dataset and identified the most frequent items and pairs.  
- Using **association rule mining**, we derived interpretable rules (X ⇒ Y) with strong **support**, **confidence**, and **lift**.  
- A light-weight recommendation evaluation showed which algorithm produced more useful rules for next-item prediction.  

**Future Work**  
- Incorporate timestamps and customer IDs (if available) to build **personalized** and **temporal** recommenders.  
- Explore **higher-order** itemsets (len ≥ 4) with careful pruning for efficiency.  
- Tune thresholds (min support/confidence) and try **advanced recommenders** (e.g., Bayesian Personalized Ranking, sequence models).  
- Deploy rules into a real-time system for **in-cart recommendations**.


## 11. About the Author / Further Resources 🧑‍💻

Hi, I'm **Taha Ali** — passionate about data science and practical AI.  
If you found this helpful, feel free to connect and explore more work:

- LinkedIn: *add your link here*  
- GitHub: *add your link here*  
- Kaggle: *add your link here*  

> Tip: Replace these placeholders with your real links to make the notebook portfolio-ready.
